<a href="https://colab.research.google.com/github/rahiakela/transformers-for-natural-language-processing/blob/main/1-model-architecture-of-the-transformer/3_architecture_of_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The architecture of Transformer

The original Transformer model is a stack of 6 layers. The output of layer $l$ is the input of layer $l+1$ until the final prediction is reached. There is a 6-layer encoder stack on the left and a 6-layer decoder stack on the right:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/transformer-architecture.png?raw=1' width='800'/>

On the left, the inputs enter the encoder side of the Transformer through an attention sub-layer and FeedForward Network (FFN) sub-layer. 

On the right, the target outputs go into the decoder side of the Transformer through two attention sub-layers and an FFN sub-layer. 

**We immediately notice that there is no RNN, LSTM, or CNN. Recurrence has been abandoned.**

Attention has replaced recurrence, which requires an increasing number of
operations as the distance between two words increases. The attention mechanism
is a "word-to-word" operation. The attention mechanism will find how each word
is related to all other words in a sequence, including the word being analyzed itself.

Let's examine the following sequence:

```
The cat sat on the mat.
```

Attention will run dot products between word vectors and determine the strongest
relationships of a word among all the other words, including itself ("cat" and "cat"):

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/attending-words.png?raw=1' width='800'/>

The attention mechanism will provide a deeper relationship between words and
produce better results.

**For each attention sub-layer, the original Transformer model runs not one but eight attention mechanisms in parallel to speed up the calculations.**

We just looked at the Transformer from the outside. Let's now go into each
component of the Transformer. We will start with the encoder.


## Setup

In [ ]:
# Transformer Installation
!pip -qq install transformers

In [ ]:
import numpy as np
from scipy.special import softmax
from transformers import pipeline

## The encoder stack

The layers of the encoder and decoder of the original Transformer model are stacks of layers. Each layer of the encoder stack has the following structure:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/encoder-stack.png?raw=1' width='800'/>

The original encoder layer structure remains the same for all of the N=6 layers of the Transformer model. Each layer contains two main sub-layers: 

- a multi-headed attention mechanism 
- a fully connected position-wise feedforward network

Notice that a residual connection surrounds each main sub-layer, $Sublayer(x)$, in the Transformer model. **These connections transport the unprocessed input $x$ of a sublayer to a layer normalization function. This way, we are certain that key information such as positional encoding is not lost on the way.**

The normalized output of each layer is thus:

$$ LayerNormalization (x + Sublayer(x)) $$

> Though the structure of each of the N=6 layers of the encoder is identical, the content of each layer is not strictly identical to the previous layer.

For example, **the embedding sub-layer is only present at the bottom level of the stack. The other five layers do not contain an embedding layer, and this guarantees that the encoded input is stable through all the layers.**

Also, **the multi-head attention mechanisms perform the same functions from layer 1 to 6. However, they do not perform the same tasks. Each layer learns from the previous layer and explores different ways of associating the tokens in the sequence.**
It looks for various associations of words, just like how we look for different
associations of letters and words when we solve a crossword puzzle.

The designers of the Transformer introduced a very efficient constraint. The output of every sub-layer of the model has a constant dimension, including the embedding layer and the residual connections. This dimension is $d_{model}$ and can be set to another value depending on your goals. **In the original Transformer architecture, $d_{model} =512$.**

**$d_{model}$ has a powerful consequence. Practically all the key operations are dot products. The dimensions remain stable, which reduces the number of operations to calculate, reduces machine consumption, and makes it easier to trace the information as it flows through the model.**

This global view of the encoder shows the highly optimized architecture of the
Transformer.

## Input embedding

## Sub-layer 1: Multi-head attention

**The multi-head attention sub-layer contains eight heads and is followed by postlayer normalization, which will add residual connections to the output of the sublayer and normalize it.**

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/sub-layer-1.png?raw=1' width='800'/>

The input of the multi-attention sub-layer of the first layer of the encoder stack is a vector that contains the embedding and the positional encoding of each word. The next layers of the stack do not start these operations over.

The dimension of the vector of each word $x_n$ of an input sequence is $d_{model} = 512$:

$$
pe(x_n) = [d_1=9.09297407e-01, d_2=9.09297407e-01, .., d_{512}=1.00000000e+00]
$$

The representation of each word $x_n$ has become a vector of $d_{model} = 512$ dimensions.

**Each word is mapped to all the other words to determine how it fits in a sequence.**

In the following sentence, we can see that "it" could be related to "cat" and "rug" in the sequence:

```
Sequence =The cat sat on the rug and it was dry-cleaned.
```

**The model will train to find out if "it" is related to "cat" or "rug."** We could run a huge calculation by training the model using the $d_{model} = 512$ dimensions as they are now.

However, we would only get one point of view at a time by analyzing the sequence
with one $d_{model}$ block. Furthermore, it would take quite some calculation time to find other perspectives.

**A better way is to divide the $d_{model} = 512$ dimensions of each word $x_n$ of $x$ (all of the words of a sequence) into $8 d_k = 64$ dimensions.**

**We then can run the 8 "heads" in parallel to speed up the training and obtain 8 different representation subspaces of how each word relates to another:**

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/multi-head-representations.png?raw=1' width='800'/>

**You can see that there are now 8 heads running in parallel.** One head might decide that "it" fits well with "cat" and another that "it" fits well with "rug" and another that "rug" fits well with "dry-cleaned."

The output of each head is a matrix $z_i$ with a shape of $x^*d_k$ The output of a multiattention head is $Z$ defined as:

$$ Z = (z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7,) $$

**However, $Z$ must be concatenated so that the output of the multi-head sub-layer is not a sequence of dimensions but one lines of $xm*d_{model}$ matrix.**

Before exiting the multi-head attention sub-layer, the elements of $Z$ are concatenated:

$$ MultiHead(output) = Concat(z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7,) = x, d_{model} $$

**Notice that each head is concatenated into $z$ that has a dimension of $d_{model} = 512$. The output of the multi-headed layer respects the constraint of the original Transformer model.**

Inside each head $h_n$ of the attention mechanism, each word vector has three
representations:

- A query vector $(Q)$ that has a dimension of $d_q = 64$, which is activated and trained when a word vector $x_n$ seeks all of the key-value pairs of the other word vectors, including itself in self-attention
- A key vector $(K)$ that has a dimension of $d_k = 64$, which will be trained to provide an attention value
- A value vector $(V)$ that has a dimension of $d_v = 64$, which will be trained to provide another attention value


Attention is defined as **Scaled Dot-Product Attention** which is represented in the following equation in which we plug $Q$, $K$ and $V$:

$$
Attention(Q,K,V) = softmax \begin{pmatrix} \frac{QK^T}{\sqrt{d_k}} \end{pmatrix} V
$$

**The vectors all have the same dimension making it relatively simple to use a scaled dot product to obtain the attention values for each head and then concatenate the output Z of the 8 heads.**

To obtain $Q$, $K$, and $V$, we must train the model with their respective weight matrices $Q_w, K_w$ and $V_w$, which have $d_k = 64$ columns and $d_{model} = 512$ rows. For example, $Q$ is obtained by a dot-product between $x$ and $Q_w. Q$ will have a dimension of $d_k = 64$.

Hugging Face and Google Brain Trax, among others, provide ready-to-use
frameworks, libraries, and modules. However, let's open the hood of the Transformer model and get our hands dirty in Python to illustrate the architecture we just explored in order to visualize the model in code and show it with intermediate images.

We will use basic Python code with only numpy and a softmax function in 10 steps to run the key aspects of the attention mechanism.

We will start by only using minimal Python functions to understand the Transformer at a low level with the inner workings of an attention head. We will explore the inner workings of the multi-head attention sub-layer using basic code.


Please refer [this notebook for implemetation Multi-head attention](https://github.com/rahiakela/transformers-for-natural-language-processing/blob/main/1-model-architecture-of-the-transformer/2_architecture_of_multi_head_attention.ipynb).







## Post-layer normalization